<a href="https://colab.research.google.com/github/vrushabhmudda/Deep-Learning-project/blob/main/Gradcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import h5py
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
import os

from google.colab import drive
drive.mount('/content/drive')

print("✅ Ready for Grad-CAM")


Mounted at /content/drive
✅ Ready for Grad-CAM


In [ ]:
# Paths
MODEL_PATH = '/content/drive/MyDrive/ISIC_2024_Project/models/inceptionv3_best.h5'
HDF5_PATH = '/content/drive/MyDrive/ISIC_2024_Project/data/raw/train-image.hdf5'
METADATA_FILE = '/content/drive/MyDrive/ISIC_2024_Project/data/raw/train-metadata.csv'
SAVE_PATH = '/content/drive/MyDrive/ISIC_2024_Project/results/gradcam/'

os.makedirs(SAVE_PATH, exist_ok=True)

# Load model
print("Loading InceptionV3...")
model = keras.models.load_model(MODEL_PATH)
print("✅ Model loaded")

# Load metadata
metadata = pd.read_csv(METADATA_FILE)
print(f"✅ Metadata: {len(metadata)} images")


Loading InceptionV3...


✅ Model loaded


/tmp/ipython-input-1500430737.py:15: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(METADATA_FILE)


✅ Metadata: 401059 images


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = keras.Model(
        inputs=[model.input],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_channel = predictions[0]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

print("✅ Grad-CAM function ready")


✅ Grad-CAM function ready


In [ ]:
print("🎨 Generating BETTER Grad-CAM visualizations...")

import cv2

# Instead of just first 5, let's select specific good examples
# Get predictions on validation set first to find clear cases

print("Finding best examples...")

# Find high-confidence malignant cases
malignant_cases = metadata[metadata['target'] == 1]
benign_cases = metadata[metadata['target'] == 0]

# Randomly sample diverse cases
malignant_sample = malignant_cases.sample(n=20, random_state=42)['isic_id'].values
benign_sample = benign_cases.sample(n=20, random_state=42)['isic_id'].values

# Quick predictions to find best examples
print("Screening candidates...")
good_malignant = []
good_benign = []

with h5py.File(HDF5_PATH, 'r') as h5_file:
    # Screen malignant
    for img_id in malignant_sample:
        jpeg_bytes = h5_file[img_id][()]
        img = Image.open(BytesIO(jpeg_bytes)).resize((299, 299))
        img_array = np.array(img, dtype=np.float32)
        if len(img_array.shape) == 2:
            img_array = np.stack([img_array] * 3, axis=-1)
        img_preprocessed = (img_array / 127.5) - 1.0
        img_batch = np.expand_dims(img_preprocessed, axis=0)
        pred = model.predict(img_batch, verbose=0)[0][0]

        # Keep high-confidence malignant (>0.7)
        if pred > 0.7:
            good_malignant.append((img_id, pred))

    # Screen benign
    for img_id in benign_sample:
        jpeg_bytes = h5_file[img_id][()]
        img = Image.open(BytesIO(jpeg_bytes)).resize((299, 299))
        img_array = np.array(img, dtype=np.float32)
        if len(img_array.shape) == 2:
            img_array = np.stack([img_array] * 3, axis=-1)
        img_preprocessed = (img_array / 127.5) - 1.0
        img_batch = np.expand_dims(img_preprocessed, axis=0)
        pred = model.predict(img_batch, verbose=0)[0][0]

        # Keep low-confidence benign (<0.3)
        if pred < 0.3:
            good_benign.append((img_id, pred))

# Sort and pick top 5 of each
good_malignant.sort(key=lambda x: x[1], reverse=True)  # Highest confidence first
good_benign.sort(key=lambda x: x[1])  # Lowest confidence first

selected_malignant = [x[0] for x in good_malignant[:5]]
selected_benign = [x[0] for x in good_benign[:5]]
sample_ids = selected_malignant + selected_benign

print(f"✅ Selected {len(selected_malignant)} malignant + {len(selected_benign)} benign cases")

# Now generate Grad-CAM for selected cases
LAST_CONV_LAYER = 'mixed10'

with h5py.File(HDF5_PATH, 'r') as h5_file:
    for i, img_id in enumerate(sample_ids):
        # Load image
        jpeg_bytes = h5_file[img_id][()]
        img = Image.open(BytesIO(jpeg_bytes)).resize((299, 299))
        img_array = np.array(img, dtype=np.float32)

        if len(img_array.shape) == 2:
            img_array = np.stack([img_array] * 3, axis=-1)

        # Preprocess
        img_preprocessed = (img_array / 127.5) - 1.0
        img_batch = np.expand_dims(img_preprocessed, axis=0)

        # Predict
        pred = model.predict(img_batch, verbose=0)[0][0]
        label = metadata[metadata['isic_id'] == img_id]['target'].values[0]

        # Grad-CAM
        heatmap = make_gradcam_heatmap(img_batch, model, LAST_CONV_LAYER)
        heatmap_resized = cv2.resize(heatmap, (299, 299))

        # Visualize
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        # Original
        axes[0].imshow(img_array.astype('uint8'))
        axes[0].set_title(f'Original\nTrue: {"Malignant" if label==1 else "Benign"}',
                         fontsize=12, fontweight='bold')
        axes[0].axis('off')

        # Heatmap
        axes[1].imshow(heatmap_resized, cmap='jet')
        axes[1].set_title('Grad-CAM Heatmap', fontsize=12, fontweight='bold')
        axes[1].axis('off')

        # Overlay
        heatmap_uint8 = np.uint8(255 * heatmap_resized)
        heatmap_colored = plt.cm.jet(heatmap_uint8 / 255.0)[:, :, :3] * 255
        overlay = heatmap_colored * 0.4 + img_array * 0.6
        overlay = overlay.astype('uint8')

        axes[2].imshow(overlay)
        axes[2].set_title(f'Overlay\nPred: {pred:.3f}', fontsize=12, fontweight='bold')
        axes[2].axis('off')

        plt.suptitle(f'{img_id}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'{SAVE_PATH}gradcam_v2_{img_id}.png', dpi=300, bbox_inches='tight')
        plt.close()

        print(f"✅ [{i+1}/10] {img_id}: True={label}, Pred={pred:.3f}")

print(f"\n✅ IMPROVED Grad-CAM images saved to:\n   {SAVE_PATH}")
print("\nThese are:")
print("  - High-confidence malignant cases (>0.7 prediction)")
print("  - Low-confidence benign cases (<0.3 prediction)")
print("  - Better visual examples for presentation!")


🎨 Generating BETTER Grad-CAM visualizations...
Finding best examples...
Screening candidates...
✅ Selected 5 malignant + 5 benign cases
✅ [1/10] ISIC_3450877: True=1, Pred=0.976
✅ [2/10] ISIC_3955107: True=1, Pred=0.963
✅ [3/10] ISIC_7231436: True=1, Pred=0.879
✅ [4/10] ISIC_6242031: True=1, Pred=0.863
✅ [5/10] ISIC_6553846: True=1, Pred=0.844
✅ [6/10] ISIC_9347557: True=0, Pred=0.016
✅ [7/10] ISIC_9413613: True=0, Pred=0.021
✅ [8/10] ISIC_9441573: True=0, Pred=0.023
✅ [9/10] ISIC_0157465: True=0, Pred=0.031
✅ [10/10] ISIC_4719164: True=0, Pred=0.036

✅ IMPROVED Grad-CAM images saved to:
   /content/drive/MyDrive/ISIC_2024_Project/results/gradcam/

These are:
  - High-confidence malignant cases (>0.7 prediction)
  - Low-confidence benign cases (<0.3 prediction)
  - Better visual examples for presentation!
